1. ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# 데이터 경로 설정
data_dir = "datasets/horse-or-human/"
img_size = (300, 300)


# 데이터셋을 불러와 정규화하는 함수
def load_and_preprocess_images(data_dir):
    images = []
    labels = []

    # 디렉토리에서 이미지와 레이블 불러오기
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0  # 정규화
                images.append(img_array)
                labels.append(0 if label_dir == "horses" else 1)  # horse: 0, human: 1

    return np.array(images), np.array(labels)


# 랜덤 회전 각도 적용을 위한 사용자 정의 함수
def random_rotation(x):
    angles = [45, 90, 135, 180, 225, 270, 315]
    angle = np.random.choice(angles)
    return tf.keras.preprocessing.image.random_rotation(x, angle)


# 이미지와 레이블 불러오기
images, labels = load_and_preprocess_images(data_dir)

# Train/Validation 데이터셋 8:2로 나누기
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# 원본 train 데이터를 추가하기 위한 ImageDataGenerator (변형 없이 사용)
original_datagen = ImageDataGenerator()

# 변형된 train 데이터를 만들기 위한 ImageDataGenerator (7가지 변형 중 랜덤 회전 포함)
augmented_datagen = ImageDataGenerator(
    preprocessing_function=random_rotation,  # 사용자 정의 회전 함수 추가
    width_shift_range=0.2,  # 가로 이동
    height_shift_range=0.2,  # 세로 이동
    shear_range=0.2,  # 찌그러뜨리기
    zoom_range=0.2,  # 크기 변형
    horizontal_flip=True,  # 좌우 반전
    fill_mode="nearest",  # 빈 픽셀 채우기
)

# 원본 train 데이터는 변형 없이 학습시키고, 변형된 데이터를 추가로 학습시킴
original_generator = original_datagen.flow(X_train, y_train, batch_size=32)
augmented_generator = augmented_datagen.flow(X_train, y_train, batch_size=32)

# 검증 데이터는 변형 없이 사용
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow(X_val, y_val, batch_size=32)

# ResNet 모델 불러오기 (사전 학습된 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# 사전 학습된 층은 훈련되지 않도록 설정
for layer in base_model.layers:
    layer.trainable = False

# 모델 구성
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))  # 이진 분류이므로 sigmoid 활성화 함수

# 모델 컴파일
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 학습 과정에서 원본 데이터와 변형된 데이터를 함께 사용
total_steps_per_epoch = (
    len(X_train) * 5 // 32
)  # 원본 + 변형된 데이터로 총 5배의 데이터셋


def combined_generator(original_gen, augmented_gen):
    while True:
        original_data = next(original_gen)  # generator에서 데이터를 가져옴
        augmented_data = next(augmented_gen)  # generator에서 데이터를 가져옴
        combined_data = np.concatenate([original_data[0], augmented_data[0]], axis=0)
        combined_labels = np.concatenate([original_data[1], augmented_data[1]], axis=0)
        yield combined_data, combined_labels


# 모델 학습
history = model.fit(
    combined_generator(original_generator, augmented_generator),
    steps_per_epoch=total_steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=len(X_val) // 32,
    epochs=5,
)

# 학습 결과 시각화
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(len(acc))

plt.figure(figsize=(10, 6))
# 정확도 시각화
plt.plot(epochs, acc, "r", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")

# 손실 시각화
plt.plot(epochs, loss, "r--", label="Training loss")
plt.plot(epochs, val_loss, "b--", label="Validation loss")

# 그래프 제목 및 레이블 설정
plt.title("Training and Validation Accuracy and Loss")
plt.xlabel("Epochs")
plt.ylabel("Accuracy / Loss")
plt.legend()

# 그래프 표시
plt.show()

In [ ]:
import cv2

# 이미지 전처리
dlist = "./datasets/test-horse-or-human/"
predict_list = os.listdir(dlist)
print(predict_list)

img = [cv2.imread(dlist + i) for i in predict_list]
img = [cv2.resize(i, (300, 300)) for i in img]
img = np.array(img)
img = img.astype("float32") / 255.0

# 분류
cutoff = 0.5
predictions = model.predict(img)
print(predictions >= cutoff)

In [ ]:
model.save("ResNet_model_test.keras")